In [1]:
from xml.dom import minidom
from collections import Counter
import glob
import os
import sys
oldstderr = sys.stderr
sys.stderr = open('logs/log_stderr.log', 'w')

import tika
from tika import detector
from tika import parser
import tabula
from tqdm import tqdm_notebook as tqdm

from utils import *

In [2]:
tika.initVM()

In [3]:
error_file = open('logs/error.log', 'w')
def parse_doc(doc):
    parsed = parser.from_file(doc, xmlContent=True)
    try:
        xmls = split_str2xml(parsed['content'])
        all_tabs = []
        for xml in xmls:
            try:
                xmldoc = minidom.parseString(xml)
                tables = xmldoc.getElementsByTagName('table')
                all_tabs+= list(tables)
            except Exception as e:
                error_file.write(f'From: {doc}')
                pass
        return all_tabs
    except KeyError as e:
        error_file.write(f'No content found for {doc}')
        return []

In [4]:
def extract_tables_pdf(pdf_doc):
    try:
        tables = tabula.read_pdf(pdf_doc, pages='all', output_format='json')
    except Exception as e:
        tables = []
        error_file.write(f'From PDF: {pdf_doc}')
    return list(map(json2xml, tables))

In [5]:
interest_type = {
    'application/vnd.oasis.opendocument.text',  # .odt
    'application/vnd.openxmlformats-officedocument.wordprocessingml.document',  #.docx
    'application/vnd.openxmlformats-officedocument.presentationml.presentation',  #.pptx
    'application/vnd.ms-powerpoint',  # .ppt
    'application/msword'  # .doc
}
all_tables = []
non_parsed_type = Counter()
for file in tqdm(glob.glob('../docs/**/*', recursive=True)):
    if not os.path.isdir(file):
        detected_type = detector.from_file(file)
        if detected_type == 'application/pdf':
            all_tables+= extract_tables_pdf(file)
        elif detected_type in interest_type:
            all_tables+= [table.toxml() for table in parse_doc(file)]
        else:
            non_parsed_type.update([detected_type])
print(non_parsed_type)  

A Jupyter Widget

No content found for ../docs/Documents/nonProcessed/05) Employee-Internal/05) ISACA/CISSP/Access_Control.ppt
No content found for ../docs/Documents/nonProcessed/05) Employee-Internal/05) ISACA/CISSP/Security_Management.ppt

Counter({'text/vnd.graphviz': 431, 'application/json': 22, 'application/zip': 17, 'application/vnd.openxmlformats-officedocument.spreadsheetml.sheet': 16, 'text/plain': 2, 'application/vnd.visio': 2, 'application/vnd.ms-excel': 2, 'application/x-tika-msoffice': 2, 'text/html': 1, 'text/x-web-markdown': 1, 'application/x-tika-ooxml-protected': 1, 'video/quicktime': 1, 'application/rtf': 1, 'image/jpeg': 1, 'application/epub+zip': 1, 'application/vnd.oasis.opendocument.presentation': 1})


In [6]:
len(all_tables)

2192